In [2]:
from dotenv import load_dotenv
import os
print("OPENAI_API_KEY: ", os.getenv("OPENAI_API_KEY")[:10])

OPENAI_API_KEY:  sk-proj-pw


# promptTemplate

In [3]:
from langchain_core.prompts import PromptTemplate


In [4]:
template = "what is a good name for a company that makes {product}?"
prompt = PromptTemplate.from_template(template)

In [6]:
formatted_prompt = prompt.format(product="colorful socks")
print(formatted_prompt)

what is a good name for a company that makes colorful socks?


# ChatOpenAI

In [7]:
from langchain_openai import ChatOpenAI


In [24]:
llm = ChatOpenAI(model="gpt-4.1-nano",max_tokens = 50, temperature=0.2)

In [9]:
response = llm.invoke([("human","Hello, how are you?")])
print(response)

content="Hello! I'm doing well, thank you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-Bk1xJDqYrPQKyQGsNChEdenO9rDZH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--26125faa-78f0-4163-b414-c8dba2f47904-0' usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Runnable
-Runnable 인터페이스 : 실행 가능 구성 요소  
-서로 다른 컴포넌트 호환되는 형태로 결합 될 수 있게 만들어 준다.

# StrOutputParser

In [10]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
parser = StrOutputParser()
parsed_responese = parser.invoke(response)
print(parsed_responese)

Hello! I'm doing well, thank you. How can I assist you today?


# Pipeline, LCEL(langChain excution language)

In [13]:
chain = prompt | llm | parser #promptTemplate -> ChatOpenAI -> StrOutpuParser

In [14]:
result = chain.invoke({"product": "coffee"})

In [15]:
print(result)

Here are some creative and appealing name ideas for a coffee company:

1. Brew Haven  
2. Java Junction  
3. Morning Mug  
4. Bean & Bliss  
5. Roast & Rise  
6. Caffeine Craft  
7. Sunrise Coffee


# LangChain 으로 토큰 사용량 추적


In [17]:
print("토큰 사용 정보 : ", response.usage_metadata)

토큰 사용 정보 :  {'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
!uv add langchain_community

Resolved 136 packages in 843ms
 Downloaded langchain-community
Prepared 14 packages in 5.29s
Installed 15 packages in 615ms
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.13
 + aiosignal==1.3.2
 + dataclasses-json==0.6.7
 + frozenlist==1.7.0
 + httpx-sse==0.4.0
 + langchain-community==0.3.25
 + marshmallow==3.26.1
 + multidict==6.5.0
 + mypy-extensions==1.1.0
 + numpy==2.3.0
 + propcache==0.3.2
 + pydantic-settings==2.9.1
 + typing-inspect==0.9.0
 + yarl==1.20.1


In [19]:
# 콜백을 통한 누적 토큰 추적
from langchain_community.callbacks import get_openai_callback

In [20]:
with get_openai_callback() as cb:
    res1 = llm.invoke("조치원의 오늘 날씨는 어떤지 알려줘")
    print("응답 : ", res1.content[:30],"....")

    res2 = llm.invoke("파이썬으로 리스트 만드는 방법을 예시로 알려줘.")
    print("응답 : ", res2.content[:30], "....")

응답 :  죄송하지만, 현재 실시간 날씨 정보를 제공할 수 없습니 ....
응답 :  물론입니다! 파이썬에서 리스트를 만드는 방법에 대해 몇 ....


In [21]:
print(f"총 토큰 수 : {cb.total_tokens}")
print(f"프롬프트(인풋) 토큰 수 : {cb.prompt_tokens}")
print(f"응답(아웃풋) 토큰 수 : {cb.completion_tokens}")
print(f"총 비용 : {cb.total_cost:.6f}")

총 토큰 수 : 130
프롬프트(인풋) 토큰 수 : 40
응답(아웃풋) 토큰 수 : 90
총 비용 : 0.000040


# LangChain LLM 응답 캐싱

In [28]:
from  langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
#InmemryCach 설정
set_llm_cache(InMemoryCache())

In [29]:
query = "재미있는  과학자 유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1 : ",result1.content)

응답1 :  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 천문학자는 항상 별을 좋아할까요?  
왜냐하면, 그들은 항상 "별"을 바라보니까요!


In [30]:
result2 = llm.invoke(query)
print("응답2 : ", result2.content)

응답2 :  물론이죠! 여기 재미있는 과학자 유머 하나 알려드릴게요:

왜 천문학자는 항상 별을 좋아할까요?  
왜냐하면, 그들은 항상 "별"을 바라보니까요!


In [33]:
# SQLite를 이용 디스크 기반 캐시 설정
import os 
from langchain_community.cache import SQLiteCache

In [34]:
# 기존 캐시 DB 삭제(실습을 위해서!)
if os.path.exists(".langchain_cache.db"):
    os.remove(".lanchain_cache.db")
    
# SQLiteCache 설정
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

In [35]:
query = "재미있는  과학자 유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1 : ", result1.content)

응답1 :  물리학자가 친구에게 말했어요, "나는 항상 힘이 세다고 생각했는데, 너는 왜 그렇게 힘이 없니?" 그러자 친구가 대답했어요, "그건 내가 힘이 없어서가 아니라,


In [36]:
result2 = llm.invoke(query)
print("응답2 : ", result2.content)

응답2 :  물리학자가 친구에게 말했어요, "나는 항상 힘이 세다고 생각했는데, 너는 왜 그렇게 힘이 없니?" 그러자 친구가 대답했어요, "그건 내가 힘이 없어서가 아니라,
